In [1]:
import pandas as pd
from sklearn import metrics
import matplotlib.pyplot as plt

In [2]:
main_data = pd.read_csv('C:/Users/ZZH/Desktop/final_main.csv')
duplicate_data = pd.read_csv('C:/Users/ZZH/Desktop/duplicate.csv')
main_data

,ID,Last Name,First Name,Date of Birth,Mother's Birth Last Name,Indigenous Identity,Religion,Ethnicity,Race,Place of Birth,Citizenship,Language (Mother Tongue),cluster_id
0,890c9866,Rael,Charles,04/30/2012,NaN,American Indian,NaN,American Indian,American Indian,NaN,Canada,NaN,10095
1,4b7c1c9f,Chase,Michael,04/01/2017,NaN,Native Hawaiian,NaN,Native Hawaiian,Native Hawaiian,NaN,Canada,English,9417
2,e056b34e,Mann,Ethan,08/07/2016,NaN,American Indian,NaN,American Indian,American Indian,NaN,Canada,Punjabi,10061
3,5d322502,Miller,Marian,04/08/2009,NaN,NaN,NaN,Black,Black,Canada,Canada,Spanish,7383
4,cb90c9e8,Jackson,James,09/17/2021,NaN,Native Hawaiian,NaN,Native Hawaiian,Native Hawaiian,NaN,Canada,English,1027
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19996,a9a00e75,Wright,Carlos,12/14/2017,NaN,NaN,Eastern Orthodoxy,NaN,Black,NaN,NaN,Vietnamese,20295
19997,72752348,Wagner,Alice,11/05/2009,NaN,NaN,NaN,East Asian,East Asian,NaN,NaN,NaN,20296
19998,20f44412,Miller,Luciano,04/08/2009,NaN,Alaska Native,NaN,Alaska Native,Alaska Native,NaN,Canada,NaN,20297
19999,70311583,Avila,Steve,06/24/2014,NaN,NaN,NaN,NaN,White,NaN,Canada,NaN,20298


In [3]:
main_data.isna().sum()

ID                              0
Last Name                       0
First Name                      0
Date of Birth                   0
Mother's Birth Last Name    19100
Indigenous Identity         10846
Religion                    14437
Ethnicity                    7605
Race                            0
Place of Birth              11950
Citizenship                  3460
Language (Mother Tongue)     9032
cluster_id                      0
dtype: int64

drop the variables which amount of NA is larger than 10000 (50%)

insight --- the contents of indigenous identity, ethnicity and race are the same.

In [4]:
data = main_data.drop(main_data.columns[[4,5,6,7,9]], axis=1)
data.fillna(value='None', inplace = True)
data.columns = ['ID', 'Last_Name', 'First_Name', 'Date_of_Birth', 'Race', 'Citizenship','Language_Mother_Tongue', 'cluster_id']
data.isna().sum()

ID                        0
Last_Name                 0
First_Name                0
Date_of_Birth             0
Race                      0
Citizenship               0
Language_Mother_Tongue    0
cluster_id                0
dtype: int64

In [5]:
# feature engineering on date
data['Date_of_Birth'] = pd.to_datetime(data['Date_of_Birth'], format = '%m/%d/%Y')
data['year'] = data['Date_of_Birth'].dt.year
data['month'] = data['Date_of_Birth'].dt.month
data['day'] = data['Date_of_Birth'].dt.day
#data.drop(columns=['Date_of_Birth'], inplace = True)
data.drop(columns=['Date_of_Birth'], inplace = True)
data.isna().sum()


ID                        0
Last_Name                 0
First_Name                0
Race                      0
Citizenship               0
Language_Mother_Tongue    0
cluster_id                0
year                      0
month                     0
day                       0
dtype: int64

In [6]:
data.dtypes

ID                        object
Last_Name                 object
First_Name                object
Race                      object
Citizenship               object
Language_Mother_Tongue    object
cluster_id                 int64
year                       int64
month                      int64
day                        int64
dtype: object

In [7]:
# combination all columns into one dataframe with 1 column 
# so that it is able to apply bagging of words
row_input = data.columns.tolist()

word_gallery = pd.DataFrame(data['ID'] + ',' + data['Last_Name'] + ',' + data['First_Name']\
                            + ',' + data['year'].astype(str) + ',' +\
                                data['month'].astype(str) + ',' + data['day'].astype(str) + ','+\
                                    data['Race'] + ','+ data['Citizenship'] + ',' \
                                    + data['Language_Mother_Tongue'], columns=['BoW'])
word_gallery


,BoW
0,"890c9866,Rael,Charles,2012,4,30,American India..."
1,"4b7c1c9f,Chase,Michael,2017,4,1,Native Hawaiia..."
2,"e056b34e,Mann,Ethan,2016,8,7,American Indian,C..."
3,"5d322502,Miller,Marian,2009,4,8,Black,Canada,S..."
4,"cb90c9e8,Jackson,James,2021,9,17,Native Hawaii..."
...,...
19996,"a9a00e75,Wright,Carlos,2017,12,14,Black,None,V..."
19997,"72752348,Wagner,Alice,2009,11,5,East Asian,Non..."
19998,"20f44412,Miller,Luciano,2009,4,8,Alaska Native..."
19999,"70311583,Avila,Steve,2014,6,24,White,Canada,None"


assign train test set

In [8]:
y_train = data['cluster_id']
X_train = word_gallery

In [9]:
# build bag of words
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()

# fit the bag of words model

X_train_count = vectorizer.fit_transform(X_train['BoW'])
X_train_count.shape

(20001, 30695)

In [10]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_count)
X_train_tfidf.shape

(20001, 30695)

In [11]:
# data preprocessing on duplicate data

duplicate_data

,ID,Last Name,First Name,Date of Birth,Mother's Birth Last Name,Indigenous Identity,Religion,Ethnicity,Race,Place of Birth,Citizenship,Language (Mother Tongue),cluster_id
0,73c2ab6a,NaN,Steven,01/13/2008,NaN,NaN,NaN,Indian,NaN,Canada,NaN,NaN,1026
1,488f1063,NaN,Dorothy,05/02/2012,NaN,NaN,Jewish,NaN,NaN,NaN,NaN,NaN,1025
2,331bea71,Yerian,Ellwood,08/18/2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,English,8465
3,ac3aaf9c,NaN,Robin,08/25/2013,NaN,NaN,NaN,Indian,NaN,Canada,NaN,English,1024
4,ae9c2ae4,NaN,Raymond,05/25/2017,NaN,NaN,Other Christianity,NaN,NaN,Canada,NaN,English,1023
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10135,08606dd8,Avila,NaN,06/24/2014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1357
10136,8507e3f4,Fernandez,NaN,08/07/2015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1356
10137,d5285223,Mathews,NaN,02/28/2015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7859
10138,7157d300,Konty,NaN,12/15/2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1355


In [12]:
ddata = duplicate_data.drop(duplicate_data.columns[[4,5,6,7,9,12]], axis=1)
ddata.fillna(value='None', inplace = True)
ddata.columns = ['ID', 'Last_Name', 'First_Name', 'Date_of_Birth', 'Race', 'Citizenship',\
                 'Language_Mother_Tongue']
ddata.isna().sum()

ID                        0
Last_Name                 0
First_Name                0
Date_of_Birth             0
Race                      0
Citizenship               0
Language_Mother_Tongue    0
dtype: int64

In [13]:
# feature engineering on date
ddata['Date_of_Birth'] = pd.to_datetime(ddata['Date_of_Birth'], format = '%m/%d/%Y')
ddata['year'] = ddata['Date_of_Birth'].dt.year
ddata['month'] = ddata['Date_of_Birth'].dt.month
ddata['day'] = ddata['Date_of_Birth'].dt.day
ddata.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10140 entries, 0 to 10139
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   ID                      10140 non-null  object        
 1   Last_Name               10140 non-null  object        
 2   First_Name              10140 non-null  object        
 3   Date_of_Birth           10140 non-null  datetime64[ns]
 4   Race                    10140 non-null  object        
 5   Citizenship             10140 non-null  object        
 6   Language_Mother_Tongue  10140 non-null  object        
 7   year                    10140 non-null  int64         
 8   month                   10140 non-null  int64         
 9   day                     10140 non-null  int64         
dtypes: datetime64[ns](1), int64(3), object(6)
memory usage: 792.3+ KB


In [14]:
word_bag = pd.DataFrame(ddata['ID'].astype(str) + ',' + ddata['Last_Name'].astype(str) +\
                              ',' + ddata['First_Name'].astype(str) +\
                              ',' + ddata['year'].astype(str) + ',' + ddata['month'].astype(str) +\
                                  ',' + ddata['day'].astype(str) + ','+\
                                      ddata['Race'].astype(str) + ','+\
                                      ddata['Citizenship'].astype(str) + ',' +\
                                          ddata['Language_Mother_Tongue'].astype(str),\
                                          columns=['BoW'])

In [15]:
X_test_count = vectorizer.transform(word_bag['BoW'])
X_test_count.shape
X_test_tfidf = tfidf_transformer.transform(X_test_count)
X_test_tfidf.shape

(10140, 30695)

In [33]:
duplicate_data.sort_values(['cluster_id'])

,ID,Last Name,First Name,Date of Birth,Mother's Birth Last Name,Indigenous Identity,Religion,Ethnicity,Race,Place of Birth,Citizenship,Language (Mother Tongue),cluster_id
1594,db41c6a1,NaN,Camilla,04/29/2022,NaN,NaN,NaN,Native Hawaiian,NaN,NaN,NaN,NaN,0
1593,dde408f0,NaN,Yolanda,05/15/2018,NaN,NaN,NaN,NaN,NaN,Canada,NaN,Tagalog,1
1592,4e82ee4a,NaN,Marion,08/28/2022,NaN,NaN,Pentecostalism and other Charismatic,NaN,NaN,Canada,NaN,Portuguese,2
1590,3eef0d37,NaN,Jesse,06/13/2016,NaN,NaN,NaN,Middle Easterner,NaN,Canada,NaN,NaN,3
1589,b2bd862f,NaN,Jean,07/09/2012,NaN,NaN,Judaism,NaN,NaN,NaN,NaN,Tagalog,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7614,4491c51e,Ohlsen,Gae,08/15/2015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10135
2532,23d9c5d9,Pty,Michele,08/07/2017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10136
8019,8cef31c8,Preiss,Baab,06/08/2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10137
4328,a2363db8,Alan,Giard,04/24/2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10138


In [16]:
y_test = duplicate_data['cluster_id']

In [34]:
'''from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(multi_class='multinomial').fit(X_train_tfidf, y_train)
pred2 = lr.predict(X_test_tfidf)
accuracy2 = metrics.accuracy_score(y_test, pred2)
print(accuracy2)'''

MemoryError: 

In [17]:
# Naive bayes
from sklearn.naive_bayes import MultinomialNB
mnb = MultinomialNB().fit(X_train_tfidf, y_train)
pred1 = mnb.predict(X_test_tfidf)
accuracy1 = metrics.accuracy_score(y_test, pred1)
print(accuracy1)

'mnb = MultinomialNB().fit(X_train_tfidf, y_train)\npred1 = mnb.predict(X_test_tfidf)\naccuracy1 = metrics.accuracy_score(y_test, pred1)\nprint(accuracy1)'

In [18]:
mnb = MultinomialNB(alpha = 0.1).fit(X_train_tfidf, y_train)
pred1 = mnb.predict(X_test_tfidf)
accuracy1 = metrics.accuracy_score(y_test, pred1)
print(accuracy1)

0.8113412228796845


In [19]:
'''mnb = MultinomialNB(alpha = 2).fit(X_train_tfidf, y_train)
pred1 = mnb.predict(X_test_tfidf)
accuracy1 = metrics.accuracy_score(y_test, pred1)
print(accuracy1)'''

'mnb = MultinomialNB(alpha = 2).fit(X_train_tfidf, y_train)\npred1 = mnb.predict(X_test_tfidf)\naccuracy1 = metrics.accuracy_score(y_test, pred1)\nprint(accuracy1)'

In [20]:
'''mnb = MultinomialNB(alpha = 0.01).fit(X_train_tfidf, y_train)
pred1 = mnb.predict(X_test_tfidf)
accuracy1 = metrics.accuracy_score(y_test, pred1)
print(accuracy1)'''

'mnb = MultinomialNB(alpha = 0.01).fit(X_train_tfidf, y_train)\npred1 = mnb.predict(X_test_tfidf)\naccuracy1 = metrics.accuracy_score(y_test, pred1)\nprint(accuracy1)'

In [21]:
'''mnb = MultinomialNB(alpha = 0.02).fit(X_train_tfidf, y_train)
pred1 = mnb.predict(X_test_tfidf)
accuracy1 = metrics.accuracy_score(y_test, pred1)
print(accuracy1)'''

'mnb = MultinomialNB(alpha = 0.02).fit(X_train_tfidf, y_train)\npred1 = mnb.predict(X_test_tfidf)\naccuracy1 = metrics.accuracy_score(y_test, pred1)\nprint(accuracy1)'

In [22]:
'''mnb = MultinomialNB(alpha = 0.2).fit(X_train_tfidf, y_train)
pred1 = mnb.predict(X_test_tfidf)
accuracy1 = metrics.accuracy_score(y_test, pred1)
print(accuracy1)'''

'mnb = MultinomialNB(alpha = 0.2).fit(X_train_tfidf, y_train)\npred1 = mnb.predict(X_test_tfidf)\naccuracy1 = metrics.accuracy_score(y_test, pred1)\nprint(accuracy1)'

In [23]:
'''mnb = MultinomialNB(alpha = 0.3).fit(X_train_tfidf, y_train)
pred1 = mnb.predict(X_test_tfidf)
accuracy1 = metrics.accuracy_score(y_test, pred1)
print(accuracy1)'''

'mnb = MultinomialNB(alpha = 0.3).fit(X_train_tfidf, y_train)\npred1 = mnb.predict(X_test_tfidf)\naccuracy1 = metrics.accuracy_score(y_test, pred1)\nprint(accuracy1)'

In [24]:
'''mnb = MultinomialNB(alpha = 0.11).fit(X_train_tfidf, y_train)
pred1 = mnb.predict(X_test_tfidf)
accuracy1 = metrics.accuracy_score(y_test, pred1)
print(accuracy1)'''

'mnb = MultinomialNB(alpha = 0.11).fit(X_train_tfidf, y_train)\npred1 = mnb.predict(X_test_tfidf)\naccuracy1 = metrics.accuracy_score(y_test, pred1)\nprint(accuracy1)'

In [25]:
'''mnb = MultinomialNB(alpha = 0.09).fit(X_train_tfidf, y_train)
pred1 = mnb.predict(X_test_tfidf)
accuracy1 = metrics.accuracy_score(y_test, pred1)
print(accuracy1)'''

'mnb = MultinomialNB(alpha = 0.09).fit(X_train_tfidf, y_train)\npred1 = mnb.predict(X_test_tfidf)\naccuracy1 = metrics.accuracy_score(y_test, pred1)\nprint(accuracy1)'

In [26]:

'''predict_confident = mnb.predict_proba(X_test_tfidf)'''


'predict_confident = mnb.predict_proba(X_test_tfidf)'

In [27]:
'''len(predict_confident[1])'''

'len(predict_confident[1])'

In [28]:
'''from sklearn.svm import SVC

svc = SVC(kernel='linear').fit(X_train_tfidf, y_train)
pred2 = svc.predict(X_test_tfidf)
accuracy2 = metrics.accuracy_score(y_test, pred2)
print(accuracy2)'''

"from sklearn.svm import SVC\n\nsvc = SVC(kernel='linear').fit(X_train_tfidf, y_train)\npred2 = svc.predict(X_test_tfidf)\naccuracy2 = metrics.accuracy_score(y_test, pred2)\nprint(accuracy2)"

In [29]:
'''from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier().fit(X_train_tfidf, y_train)
pred3 = mlp.predict(X_test_tfidf)
accuracy3 = metrics.accuracy_score(y_test, pred3)
print(accuracy3)'''

'from sklearn.neural_network import MLPClassifier\n\nmlp = MLPClassifier().fit(X_train_tfidf, y_train)\npred3 = mlp.predict(X_test_tfidf)\naccuracy3 = metrics.accuracy_score(y_test, pred3)\nprint(accuracy3)'

In [30]:
# tune the parameters by BayesianOptimization     RandomizedSearchCV,GridSearchCV---can not use because there is only 1 sample in each class, there should be at least 2 samples in each class (n_splits = 2)
'''from bayes_opt import BayesianOptimization
from scipy.stats import uniform
# define values to search
pbounds = {'alpha': (0.1, 1, 2, 3)}
# define the grid search procedure
mnb_tuning = BayesianOptimization(f=evaluate, pbounds=pbounds, n_jobs = -1)
mnb_tuning.fit(X_train_tfidf, y_train)
print(mnb_tuning.best_params_)'''

"from bayes_opt import BayesianOptimization\nfrom scipy.stats import uniform\n# define values to search\npbounds = {'alpha': (0.1, 1, 2, 3)}\n# define the grid search procedure\nmnb_tuning = BayesianOptimization(f=evaluate, pbounds=pbounds, n_jobs = -1)\nmnb_tuning.fit(X_train_tfidf, y_train)\nprint(mnb_tuning.best_params_)"

In [31]:
'''final_pred = mnb.predict(X_test_tfidf)
accuracy_final = metrics.accuracy_score(y_test, final_pred)
print(accuracy1)'''

'final_pred = mnb.predict(X_test_tfidf)\naccuracy_final = metrics.accuracy_score(y_test, final_pred)\nprint(accuracy1)'